# Process SmolLM Corpus

In [ ]:
from datasets import load_dataset
import boto3
import gzip,os

# cv2 = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2")
# fed = load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup")
# pe = load_dataset("HuggingFaceTB/smollm-corpus", "python-edu")

# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed4 = fed.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=120)
# fed4=load_dataset("chengjunyan1/smollm-10", "fineweb-edu-dedup")

In [ ]:
pe4 = pe.filter(lambda x: x['score'] > 4.16, num_proc=120)
pe4['train'].num_rows,pe4['train'].num_rows/pe['train'].num_rows

In [ ]:
from datasets import dataset_dict

cv2_ratio=(fed4['train'].num_rows/fed['train'].num_rows+pe4['train'].num_rows/pe['train'].num_rows)/2
cv2_samples=int(cv2.num_rows['train']*cv2_ratio)
cv2_10=cv2['train'].shuffle(seed=42).select(range(cv2_samples))
cv2_10=dataset_dict.DatasetDict({'train':cv2_10})

In [ ]:
len_all=fed4['train'].num_rows+pe4['train'].num_rows+cv2_10['train'].num_rows
len_all_full=fed['train'].num_rows+pe['train'].num_rows+cv2['train'].num_rows
print(fed4['train'].num_rows/len_all, pe4['train'].num_rows/len_all, cv2_10['train'].num_rows/len_all)
print(fed['train'].num_rows/len_all_full, pe['train'].num_rows/len_all_full, cv2['train'].num_rows/len_all_full)

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# pe4.push_to_hub("chengjunyan1/smollm-10","python-edu")
# fed4.push_to_hub("chengjunyan1/smollm-10","fineweb-edu-dedup")
# cv2_10.push_to_hub("chengjunyan1/smollm-10","cosmopedia-v2")

# Verify processed corpus

In [1]:
from datasets import load_dataset
import boto3
import gzip,os
import functools as ft


# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup")
cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2")

In [2]:
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

pe10=download_python_edu()

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Parameter 'function'=<function download_contents at 0x7f6cbbdc0d60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=512):   0%|          | 0/864074 [00:00<?, ? examples/s]

TimeoutError: 

In [16]:
pe10['train'][100]

{'blob_id': '55abeaf992065b49348262fd31cc660a1f68c3ea',
 'repo_name': 'Blaine-Rob5000/STECH-Capstone',
 'path': '/whiteboard.py',
 'length_bytes': 7617,
 'score': 4.34375,
 'int_score': 4,
 'text': '"""Whiteboard projects\r\n\r\n   Potential whiteboard project questions\r\n\r\n       created\r\n           by:  R. G. Blaine\r\n           on:  April, 9, 2018\r\n"""\r\n\r\n# imports\r\nimport random\r\n\r\n\r\n################################################################################\r\n\r\ndef listFibs(n):\r\n\r\n    \'\'\'Creates and prints a list of the first n fibonacci numbers.\r\n\r\n    arguments:\r\n        n : integer 2 or greater\r\n\r\n    returns:\r\n        none\r\n    \'\'\'\r\n\r\n    # variables\r\n    fibNums = []            # list to hold the fibonacci numbers\r\n    fibNums.append(0)       # the 1st fibonacci number\r\n    fibNums.append(1)       # the 2nd fibonacci number\r\n\r\n    # error handling\r\n    if (n < 2) or (n != int(n)):\r\n        print("\\n\\n\\n 

In [ ]:
from datasets import concatenate_datasets,DatasetDict
import numpy as np


def resample_dataset(dataset, weight):
    num_samples = int(len(dataset) * weight)
    indices = np.random.choice(len(dataset), num_samples, replace=True)
    return dataset.select(indices)

def combine_datasets(dataset_dicts, weights:dict=None): # weights e.g. {'train':[1.5,1.0]}
    combined_dict = {}
    
    # Initialize weights if not provided
    for dataset_dict in dataset_dicts:
        for key, dataset in dataset_dict.items():
            if key in combined_dict:
                combined_dict[key] = concatenate_datasets([combined_dict[key], dataset])
            else:
                combined_dict[key] = dataset

    # Apply weights by resampling the datasets
    for key in combined_dict:
        datasets = []
        for idx,dataset in enumerate(dataset_dicts):
            if key in dataset:
                if weights is None or key not in weights or weights[key][idx] == 1.0:
                    datasets.append(dataset[key])
                else:
                    resampled_dataset = resample_dataset(dataset[key], weights[key][idx])
                    datasets.append(resampled_dataset)
        combined_dict[key] = concatenate_datasets(datasets)
    
    return DatasetDict(combined_dict)


dataset_dicts=[pe10,fed10,cv10]
combined=combine_datasets(dataset_dicts)

In [ ]:
fed10['train'][0]